In [3]:
import pandas as pd
from splink.datasets import splink_datasets
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl

df = splink_datasets.historical_50k

In [5]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on(["first_name", "surname"]),
        block_on(["surname", "dob"]),
        block_on(["first_name", "dob"]),
        block_on(["postcode_fake", "first_name"]),
    ],
    "comparisons": [
        ctl.name_comparison("first_name", term_frequency_adjustments=True),
        ctl.name_comparison("surname", term_frequency_adjustments=True),
        ctl.date_comparison("dob", cast_strings_to_date=True, invalid_dates_as_null=True),
        ctl.postcode_comparison("postcode_fake"),
        cl.exact_match("birth_place", term_frequency_adjustments=True),
        cl.exact_match("occupation",  term_frequency_adjustments=True),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "max_iterations": 10,
    "em_convergence": 0.01
}

linker = DuckDBLinker(df, settings)

linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

linker.estimate_u_using_random_sampling(max_pairs=5e6)

training_blocking_rule = block_on(["first_name", "surname"])
training_session_names = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = block_on("dob")
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

linker.save_model_to_json("model_h50k.json", overwrite=True)

Probability two random records match is estimated to be  0.000136.
This means that amongst all possible pairwise record comparisons, one in 7,362.31 are expected to match.  With 1,279,041,753 total possible comparisons, we expect a total of around 173,728.33 matching pairs
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - postcode_fake (no m values are trained).
    - birth_place (no m values are trained).
    - occupation (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupat

{'link_type': 'dedupe_only',
 'blocking_rules_to_generate_predictions': [{'blocking_rule': '(l."first_name" = r."first_name") AND (l."surname" = r."surname")',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': '(l."surname" = r."surname") AND (l."dob" = r."dob")',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': '(l."first_name" = r."first_name") AND (l."dob" = r."dob")',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': '(l."postcode_fake" = r."postcode_fake") AND (l."first_name" = r."first_name")',
   'sql_dialect': 'duckdb'}],
 'comparisons': [{'output_column_name': 'first_name',
   'comparison_levels': [{'sql_condition': '"first_name_l" IS NULL OR "first_name_r" IS NULL',
     'label_for_charts': 'Null',
     'is_null_level': True},
    {'sql_condition': '"first_name_l" = "first_name_r"',
     'label_for_charts': 'Exact match first_name',
     'm_probability': 0.5441309704828743,
     'u_probability': 0.012224508462954514,
     'tf_adjustment_column': 'first_name',
     'tf_adjustmen